https://adventofcode.com/2022/day/15

In [1]:
import re
import functools
from math import inf

import numpy as np
import itertools as it

Point = tuple[int, int]
Sensor = tuple[int, int]
Beacon = tuple[int, int]


def manhattan(p0: Point, p1: Point) -> int:
    return sum(abs(first - second) for first, second in zip(p0, p1))


def parseInput(input_file: str) -> tuple[list[Sensor], list[Beacon]]:
    with open(input_file) as f:
        lines = f.readlines()

    sensors, beacons, distances = list(), list(), list()
    for l in lines:
        xs, ys, xb, yb = map(
            int, re.search(r"x=(-?\d+),\s+y=(-?\d+).*x=(-?\d+),\s+y=(-?\d+)", l).groups()
        )
        sensors.append((xs, ys))
        beacons.append((xb, yb))
        distances.append(manhattan((xs, ys), (xb, yb)))

    return (sensors, beacons, distances)


def computeSensorXRanges(sensors: list[Sensor], distances: list[int], y: int) -> list[range]:
    ranges = list()
    for s, d in zip(sensors, distances):
        xOffset = d - manhattan((s[0], y), s)
        if xOffset >= 0:
            ranges.append(range(s[0] - xOffset, s[0] + xOffset + 1))

    return ranges


def getLargestContainingRange(x: int, ranges: list[range]) -> range:
    lrange, maxLen = None, 0
    for r in ranges:
        rLen = len(r)
        if (x in r) and (rLen > maxLen):
            lrange = r
            maxLen = rLen

    return lrange


def solvePart1(sensors: list[Sensor], beacons: list[Beacon], distances: list[int], y: int) -> int:
    ranges = computeSensorXRanges(sensors, distances, y)
    rangeUnion = functools.reduce(lambda a, b: a | set(b), ranges, set())
    xMin, xMax = min(rangeUnion), max(rangeUnion)

    row = np.ones(xMax - xMin + 1, dtype=int)
    for x in rangeUnion ^ set(range(xMin, xMax + 1)):
        row[x - xMin] = 0
    for x, _ in filter(lambda b: b[1] == y, beacons):
        row[x - xMin] = 0

    return sum(row)


def solvePart2(
    sensors: list[Sensor], beacons: list[Beacon], distances: list[int], xyLim: int
) -> int:
    x, y, xf, yf = 0, 0, 0, 0
    found = False
    for y in range(xyLim + 1):
        ranges = computeSensorXRanges(sensors, distances, y)
        x = 0
        while x <= xyLim:
            r = getLargestContainingRange(x, ranges)
            if r is None:
                rangeUnion = functools.reduce(lambda a, b: a | set(b), ranges, set())
                maxXRange = set(range(xyLim + 1))
                xf, yf = tuple(rangeUnion ^ maxXRange)[0], y
                found = True
                break
            x = max(x + 1, r[-1])
        if found:
            break

    return xf * 4_000_000 + yf


In [2]:
sensors, beacons, distances = parseInput("test_input.txt")

result, expected = solvePart1(sensors, beacons, distances, 10), 26
assert result == expected, f"Part 1: {result=} is wrong ({expected=})"

result, expected = solvePart2(sensors, beacons, distances, 20), 56000011
assert result == expected, f"Part 2: {result=} is wrong ({expected=})"


In [3]:
sensors, beacons, distances = parseInput("input.txt")

print(solvePart1(sensors, beacons, distances, 2_000_000))
print(solvePart2(sensors, beacons, distances, 4_000_000))

5112034
13172087230812
